In [1]:
import requests
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import datetime

load_dotenv(find_dotenv())

client = OpenAI()

def get_opening_hours():
    try:
        response = requests.get('http://localhost:8000/opening-hours')
        if response.status_code == 200:
            return response.json()  # Assuming the API returns JSON data
        else:
            return "Error: Unable to fetch opening hours."
    except requests.exceptions.RequestException as e:
        return f"Request failed: {e}"

def how_late():
    return datetime.datetime.now().strftime("%H:%M:%S")  # Returns current time in HH:MM:SS format

file = client.files.create(
    file=open("knowledge.txt", "rb"),
    purpose='assistants'
)

In [2]:
assistant = client.beta.assistants.create(
    name="Summariser",
    description="You are great at assisting customers of a bank.",
    model="gpt-4-turbo-preview",
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {
            "type": "function",
            "function": {
                "name": "get_opening_hours",
                "description": "Get the opening hours of the bank",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "how_late",
                "description": "Get the current time",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }
    ],
    file_ids=[file.id]
)
print(assistant)

Assistant(id='asst_6Ek4jHcMW37mP6idM7is0EQN', created_at=1706817404, description='You are great at assisting customers of a bank.', file_ids=['file-vuFVgAjTRqgnnIHroDa6cdtX'], instructions=None, metadata={}, model='gpt-4-turbo-preview', name='Summariser', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter'), ToolRetrieval(type='retrieval'), ToolFunction(function=FunctionDefinition(name='get_opening_hours', description='Get the opening hours of the bank', parameters={'type': 'object', 'properties': {}, 'required': []}), type='function'), ToolFunction(function=FunctionDefinition(name='how_late', description='Get the current time', parameters={'type': 'object', 'properties': {}, 'required': []}), type='function')])


In [3]:
thread = client.beta.threads.create()

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Wie viel kostet ein Darlehen pro Jahr?"
)

In [5]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  tools=[{"type": "code_interpreter"}, {"type": "retrieval"}] # allow to override assistant tools
)

In [6]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(run)

Run(id='run_tD5vsmPfZMRHq7x0A4wa8yXs', assistant_id='asst_6Ek4jHcMW37mP6idM7is0EQN', cancelled_at=None, completed_at=None, created_at=1706817405, expires_at=1706818005, failed_at=None, file_ids=['file-vuFVgAjTRqgnnIHroDa6cdtX'], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-4-turbo-preview', object='thread.run', required_action=None, started_at=1706817406, status='in_progress', thread_id='thread_vp9D5mLGSI2h0Eqw5wTqZ0wX', tools=[ToolAssistantToolsCode(type='code_interpreter'), ToolAssistantToolsRetrieval(type='retrieval')], usage=None)


Polling is required the get the current status

In [7]:
import time

def get_run_status(client, thread_id, run_id):
    try:
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        return {"status": run.status, "run": run}
    except Exception as e:
        print(f"Error retrieving run status: {e}")
        return {"status": None, "run": None}

def poll_run_status(client, thread_id, run_id):
    while True:
        result = get_run_status(client, thread_id, run_id)
        status = result["status"]
        run = result["run"]
        if status in ['completed', 'cancelled', 'failed', 'requires_action']:
            print(f"Run has finished or requires action with status: {status}")
            break
        else:
            print("Run is not finished yet...")
            time.sleep(2)

    return {"status": status, "run": run}

In [8]:
result = poll_run_status(client=client, thread_id=thread.id, run_id=run.id)

Run is not finished yet...
Run is not finished yet...
Run is not finished yet...
Run is not finished yet...
Run is not finished yet...
Run is not finished yet...
Run is not finished yet...
Run is not finished yet...
Run is not finished yet...
Run is not finished yet...
Run is not finished yet...
Run has finished or requires action with status: completed


In [9]:
print(result["run"])

Run(id='run_tD5vsmPfZMRHq7x0A4wa8yXs', assistant_id='asst_6Ek4jHcMW37mP6idM7is0EQN', cancelled_at=None, completed_at=1706817429, created_at=1706817405, expires_at=None, failed_at=None, file_ids=['file-vuFVgAjTRqgnnIHroDa6cdtX'], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, metadata={}, model='gpt-4-turbo-preview', object='thread.run', required_action=None, started_at=1706817406, status='completed', thread_id='thread_vp9D5mLGSI2h0Eqw5wTqZ0wX', tools=[ToolAssistantToolsCode(type='code_interpreter'), ToolAssistantToolsRetrieval(type='retrieval')], usage=Usage(completion_tokens=325, prompt_tokens=1461, total_tokens=1786))


In [10]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print(messages.data)

[ThreadMessage(id='msg_OkvJleYBdqAxsAWx1Eh9y1cP', assistant_id='asst_6Ek4jHcMW37mP6idM7is0EQN', content=[MessageContentText(text=Text(annotations=[], value='Die Kosten eines Darlehens pro Jahr variieren je nach Art des Darlehens. Hier sind einige Beispiele aus dem Dokument:\n\n- **Persönliche Darlehen**: Ab 4,5% effektiver Jahreszins (APR).\n- **Hypothekenoptionen**:\n  - **30-jährige Festhypothek**: Ab 3,75%.\n  - **Programme für Erstkäufer**: Diese bieten niedrigere Anzahlungen und Hilfe bei Abschlusskosten. Der genaue APR wird nicht erwähnt.\n  - **Jumbo-Hypotheken**: Maßgeschneiderte Zinssätze und Konditionen für hochwertige Immobilien. Auch hier wird der genaue APR nicht angegeben.\n- **Gewerbliche Kredite**: Wettbewerbsfähige Zinssätze und flexible Bedingungen. der genaue APR wird nicht angegeben.\n- **Kredit-Aufbau-Programm**:\n  - **Gesicherte Kreditkarte**: Bietet eine niedrige Jahresgebühr und einen wettbewerbsfähigen APR. Der genaue Wert wird jedoch nicht angegeben.\n\nDaher

In [11]:
for message in messages.data:
    if message.content and message.content[0].text:
        print(f"Message: {message.content[0].text.value}")

Message: Die Kosten eines Darlehens pro Jahr variieren je nach Art des Darlehens. Hier sind einige Beispiele aus dem Dokument:

- **Persönliche Darlehen**: Ab 4,5% effektiver Jahreszins (APR).
- **Hypothekenoptionen**:
  - **30-jährige Festhypothek**: Ab 3,75%.
  - **Programme für Erstkäufer**: Diese bieten niedrigere Anzahlungen und Hilfe bei Abschlusskosten. Der genaue APR wird nicht erwähnt.
  - **Jumbo-Hypotheken**: Maßgeschneiderte Zinssätze und Konditionen für hochwertige Immobilien. Auch hier wird der genaue APR nicht angegeben.
- **Gewerbliche Kredite**: Wettbewerbsfähige Zinssätze und flexible Bedingungen. der genaue APR wird nicht angegeben.
- **Kredit-Aufbau-Programm**:
  - **Gesicherte Kreditkarte**: Bietet eine niedrige Jahresgebühr und einen wettbewerbsfähigen APR. Der genaue Wert wird jedoch nicht angegeben.

Daher variiert der jährliche Kosten eines Darlehens je nach Typ und den spezifischen Konditionen des Darlehens.
Message: Wie viel kostet ein Darlehen pro Jahr?


### Function usage

In [12]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Wie spät ist es?"
)

In [13]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
)

In [14]:
run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

In [15]:
result = poll_run_status(client=client, thread_id=thread.id, run_id=run.id)

Run is not finished yet...
Run has finished or requires action with status: requires_action


In [16]:
run.required_action.submit_tool_outputs.tool_calls

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

Now we call our tool

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [ ]:
messages

### Delete everthing after using it!

In [ ]:
# deleted_assistant_file = client.beta.assistants.files.delete(
#     assistant_id=assistant.id,
#     file_id=file.id
# )
# print(deleted_assistant_file)

# response = client.beta.assistants.delete(assistant.id)
# print(response)